**PORORO pretrained model**

In [ ]:
# pororo only supports python>=3.6
!python -V

Python 3.6.0


In [ ]:
!wget https://www.python.org/ftp/python/3.6.0/Python-3.6.0.tgz
!tar xvfz Python-3.6.0.tgz
!Python-3.6.0/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.6/test/test_set.py'...
Compiling '/usr/local/lib/python3.6/test/test_setcomps.py'...
Compiling '/usr/local/lib/python3.6/test/test_shelve.py'...
Compiling '/usr/local/lib/python3.6/test/test_shlex.py'...
Compiling '/usr/local/lib/python3.6/test/test_shutil.py'...
Compiling '/usr/local/lib/python3.6/test/test_signal.py'...
Compiling '/usr/local/lib/python3.6/test/test_site.py'...
Compiling '/usr/local/lib/python3.6/test/test_slice.py'...
Compiling '/usr/local/lib/python3.6/test/test_smtpd.py'...
Compiling '/usr/local/lib/python3.6/test/test_smtplib.py'...
Compiling '/usr/local/lib/python3.6/test/test_smtpnet.py'...
Compiling '/usr/local/lib/python3.6/test/test_sndhdr.py'...
Compiling '/usr/local/lib/python3.6/test/test_socket.py'...
Compiling '/usr/local/lib/python3.6/test/test_socketserver.py'...
Compiling '/usr/local/lib/python3.6/test/test_sort.py'...
Compiling '/usr/local/lib/python3.6/test/test_source_encoding.py

In [ ]:
!pip install pororo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 KB 5.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/kakaobrain/pororo.git
!cd pororo
!pip install -e .

fatal: destination path 'pororo' already exists and is not an empty directory.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

test_data = pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/test_data.csv')
submission_data = pd.read_csv("/content/drive/MyDrive/한국어문장분류공모전/DATA/sample_submission.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
premise, hypothesis = list(test_data['premise']), list(test_data['hypothesis'])

In [ ]:
from pororo import Pororo
Pororo.available_tasks()
nli_module = Pororo(task="nli", lang="ko")
nli_module(premise[0], hypothesis[0])

'Contradiction'

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm_notebook

nli_results = []

for pre, hypo in tqdm_notebook(zip(premise, hypothesis)):
  result = nli_module(pre, hypo)

  nli_results.append(result)


In [ ]:
submission_data['label'] = nli_results
submission_data.head(5)

,index,label
0,0,Contradiction
1,1,Entailment
2,2,Entailment
3,3,Contradiction
4,4,Entailment


In [ ]:
import numpy as np

def label_to_num(label):
    label_dict = {"Entailment":0, "entailment":0, "Contradiction":1, "contradiction":1, "Neutral":2, "neutral":2}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label


In [ ]:
answer = np.array(label_to_num(submission_data['label']))
answer_pororo = np.array(range(1666))
for i in range(0,1666):
  answer_pororo[i] = answer[i][1]
print(answer_pororo[0:10])

**Huffon/klue-roberta-base-nli pretrained model**

In [ ]:
from transformers import pipeline, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
model = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=False)

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/926 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/test_data.csv')
X_test = test_data["premise"] + " " + tokenizer.sep_token + " " + test_data["hypothesis"]

In [ ]:
from tqdm import tqdm
y_preds = []
for i in tqdm(range(X_test.shape[0])):
    y_pred = model(X_test[i])[0]["label"].lower()
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

100%|██████████| 1666/1666 [05:21<00:00,  5.18it/s]


In [ ]:
submission_data["label"] = y_preds
submission = submission_data.loc[:, ["index", "label"]]

In [ ]:
answer = np.array(label_to_num(submission['label']))
answer_Huffon = np.array(range(1666))
for i in range(0,1666):
  answer_Huffon[i] = answer[i][1]
print(answer_Huffon[0:10])

[1 2 0 1 0 2 2 0 0 2]


**Fine-tuned Huggingface roberta-large 모델 결과파일 로드**

In [ ]:
df_roberta_large = pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/Cornsomazing0225_2.csv')
answer = np.array(label_to_num(df_roberta_large['label']))
answer_roberta_large = np.array(range(1666))
for i in range(0,1666):
  answer_roberta_large[i] = answer[i][1]
print(answer_roberta_large[0:10])


[1 0 0 1 1 2 2 2 0 1]


**Fine-tuned Huggingface bert-base 모델 결과파일 로드**

In [ ]:
df_TPU = pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/result_TPU.csv')
answer = np.array(label_to_num(df_TPU['label']))
answer_TPU = np.array(range(1666))
for i in range(0,1666):
  answer_TPU[i] = answer[i][1]
print(answer_TPU[0:10])

[1 2 0 1 1 2 2 0 0 2]


**Ensemble**

In [ ]:
print(answer_pororo.shape)
print(answer_Huffon.shape)
print(answer_roberta_large.shape)
print(answer_TPU.shape)
answer_final = np.array([answer_pororo, answer_Huffon, answer_roberta_large, answer_TPU])
print(answer_final.shape)

(1666,)
(1666,)
(1666,)
(1666,)
(4, 1666)


In [ ]:
submission_np = np.array(range(1666))
for i in range(1666):
  vals,counts = np.unique([answer_final[0][i], answer_final[1][i], answer_final[2][i], answer_final[3][i]], return_counts=True) 
  index = np.argmax(counts) 
  submission_np[i] = vals[index]

submission_np[0:10]

array([1, 0, 0, 1, 0, 2, 2, 0, 0, 1])

In [ ]:
def num_to_label(num):
    label_dict = {0:"entailment", 1:"contradiction", 2:"neutral"}
    str_label = []

    for i, v in enumerate(num):
        str_label.append([i,label_dict[v]])
    
    return str_label

In [ ]:
submission_sample = num_to_label(submission_np)

In [ ]:
submission_sample[0:10]

[[0, 'contradiction'],
 [1, 'entailment'],
 [2, 'entailment'],
 [3, 'contradiction'],
 [4, 'entailment'],
 [5, 'neutral'],
 [6, 'neutral'],
 [7, 'entailment'],
 [8, 'entailment'],
 [9, 'contradiction']]

In [ ]:
df = pd.DataFrame(submission_sample, columns=['index', 'label'])

df.to_csv('./Cornsomazing0228.csv', index=False)

print(df)

      index          label
0         0  contradiction
1         1     entailment
2         2     entailment
3         3  contradiction
4         4     entailment
...     ...            ...
1661   1661        neutral
1662   1662     entailment
1663   1663        neutral
1664   1664        neutral
1665   1665        neutral

[1666 rows x 2 columns]
